In [1]:
!pip install transformers

# Import MLM model and tokenizer

In [2]:
from transformers import BertTokenizer, BertForMaskedLM

In [3]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Test

In [4]:
model.eval() # model to eval mode
text = 'I want to [MASK] the car because it is cheap . Malaria is a [MASK] disease.'
ids =  tokenizer.encode(text, return_tensors='pt')

In [5]:
ids

tensor([[  101,  1045,  2215,  2000,   103,  1996,  2482,  2138,  2009,  2003,
         10036,  1012, 19132,  2003,  1037,   103,  4295,  1012,   102]])

In [6]:
import torch
mask_token_index = torch.where(ids == tokenizer.mask_token_id)[1]
mask_token_index

tensor([ 4, 15])

In [7]:
with torch.no_grad():
    output = model(ids)

In [8]:
mask_token_logits = output.logits[0, mask_token_index, :]

In [9]:
torch.topk(mask_token_logits, 5, dim=1).indices.tolist()

[[5271, 2202, 4965, 3298, 9278], [3809, 2691, 9252, 5133, 5536]]

In [10]:
top5s = torch.topk(mask_token_logits, 5, dim=1).indices.tolist()
for top5 in top5s:
  predict_words = [tokenizer.decode([id]) for id in top5]
  print(predict_words)


['sell', 'take', 'buy', 'drive', 'rent']
['serious', 'common', 'deadly', 'tropical', 'terminal']


# Test on vnmese

In [11]:
!pip3 install py_vncorenlp

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 22.4 MB/s eta 0:00:00
  Created wheel for py_vncorenlp: filename=py_vncorenlp-0.1.4-py3-none-any.whl size=4305 sha256=4e835e6417803a7cdc7c63f0db6f4bc01e0fa20dd318c742121212c6ff7acd91
  Stored in directory: /root/.cache/pip/wheels/d5/d9/bf/62632cdb007c702a0664091e92a0bb1f18a2fcecbe962d9827
Successfully built py_vncorenlp


In [12]:
!mkdir vncorenlp

In [13]:
import py_vncorenlp
py_vncorenlp.download_model(save_dir='/content/vncorenlp')
model = py_vncorenlp.VnCoreNLP(save_dir='/content/vncorenlp')


In [14]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='bert-base-multilingual-cased')

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [15]:
sentence = "Ngày hôm nay, có một cô gái [MASK] và làm toán với tôi. Tôi rất [MASK]"
def masked_word_seg(sentence):
  texts = model.word_segment(sentence)
  texts = [text.replace('[ MASK ]', '[MASK]') for text in texts]
  return " ".join(texts)

seg_sentence = masked_word_seg(sentence)

In [16]:
unmasker(seg_sentence)

[[{'score': 0.19624048471450806,
   'token': 12002,
   'token_str': 'đến',
   'sequence': '[CLS] Ngày hôm _ nay, có một cô gái đến và làm toán với tôi. Tôi rất [MASK] [SEP]'},
  {'score': 0.06345025449991226,
   'token': 117,
   'token_str': ',',
   'sequence': '[CLS] Ngày hôm _ nay, có một cô gái, và làm toán với tôi. Tôi rất [MASK] [SEP]'},
  {'score': 0.04962209612131119,
   'token': 16895,
   'token_str': 'đi',
   'sequence': '[CLS] Ngày hôm _ nay, có một cô gái đi và làm toán với tôi. Tôi rất [MASK] [SEP]'},
  {'score': 0.03792055696249008,
   'token': 22072,
   'token_str': 'nói',
   'sequence': '[CLS] Ngày hôm _ nay, có một cô gái nói và làm toán với tôi. Tôi rất [MASK] [SEP]'},
  {'score': 0.03616345673799515,
   'token': 43094,
   'token_str': 'bạn',
   'sequence': '[CLS] Ngày hôm _ nay, có một cô gái bạn và làm toán với tôi. Tôi rất [MASK] [SEP]'}],
 [{'score': 0.41026225686073303,
   'token': 119,
   'token_str': '.',
   'sequence': '[CLS] Ngày hôm _ nay, có một cô gái [MASK

# Training

In [17]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained("bert-base-uncased")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
!mkdir ../data

In [19]:
!curl -o ../data/cleandata.txt https://raw.githubusercontent.com/jamescalam/transformers/main/data/text/meditations/clean.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  235k  100  235k    0     0   488k      0 --:--:-- --:--:-- --:--:--  490k


In [20]:
with open("../data/cleandata.txt") as f:
  texts = f.read().split("\n");

texts[:3]

['From my grandfather Verus I learned good morals and the government of my temper.',
 'From the reputation and remembrance of my father, modesty and a manly character.',
 'From my mother, piety and beneficence, and abstinence, not only from evil deeds, but even from evil thoughts; and further, simplicity in my way of living, far removed from the habits of the rich.']

In [21]:
tokenized_inputs = tokenizer(texts,return_tensors="pt",max_length=512, truncation=True,padding="max_length")

In [22]:
tokenized_inputs['labels'] = tokenized_inputs.input_ids.detach().clone()
tokenized_inputs

{'input_ids': tensor([[  101,  2013,  2026,  ...,     0,     0,     0],
        [  101,  2013,  1996,  ...,     0,     0,     0],
        [  101,  2013,  2026,  ...,     0,     0,     0],
        ...,
        [  101,  3459,  2185,  ...,     0,     0,     0],
        [  101,  2043, 15223,  ...,     0,     0,     0],
        [  101,  7887,  3288,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[  101,  2013,  2026,  ...,     0,     0,     0],
        [  101,  2013,  1996,  ...,     0,     0,     0],
        [  101,  2013, 

In [23]:
input_ids = tokenized_inputs.input_ids

In [24]:
import torch

In [25]:
rand = torch.rand(input_ids.shape)

In [26]:
(rand<0.15).sum()/(rand.size()[0]*rand.size()[1])
# choosing 0.15 ask bert paper

tensor(0.1509)

In [27]:
masked_matrix = (rand<0.15) & (input_ids!=101) & (input_ids!=102) & (input_ids!=0)

In [28]:
masked_input = torch.where(masked_matrix, torch.full_like(input_ids, 103), input_ids)


In [29]:
tokenized_inputs.input_ids = masked_input

In [30]:
from torch.utils.data import DataLoader,Dataset

In [31]:
class MeditationDataset(Dataset):
  def __init__(self, encodings):
    self.encodings = encodings
  def __getitem__(self,idx):
    return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
  def __len__(self):
    return len(self.encodings.input_ids)


In [32]:
dataset = MeditationDataset(tokenized_inputs)

In [33]:
dataloader = DataLoader(dataset,batch_size=16,shuffle= True)

In [34]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device("cpu")

In [35]:
device

device(type='cuda')

In [36]:
from tqdm import tqdm
from transformers import AdamW

In [37]:
optim = AdamW(model.parameters(),lr=1e-5)
epochs = 2

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [43]:
input_ids

tensor([[  101,  1997,  2529,  ...,     0,     0,     0],
        [  101,  9530, 18532,  ...,     0,     0,     0],
        [  101,  4088,  1996,  ...,     0,     0,     0],
        ...,
        [  101,  2074,  2004,  ...,  2373,  2029,   102],
        [  101,  2002,  2040,  ...,     0,     0,     0],
        [  101,  2013,  2026,  ...,     0,     0,     0]], device='cuda:0')

In [44]:
model = model.to(device)
for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(dataloader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/32 [00:00<?, ?it/s]<ipython-input-31-f38724a228bf>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Epoch 1: 100%|██████████| 32/32 [01:01<00:00,  1.93s/it, loss=0.959]
